In [2]:
import pandas as pd
import pyspark.sql.functions as F
from datetime import datetime
from pyspark.sql.types import *
from pyspark.storagelevel import StorageLevel
import numpy as np

pd.set_option("display.max_rows", 101)
pd.set_option("display.max_columns", 101)

In [3]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
# from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler

from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

from pyspark.sql import Row
from pyspark.ml.linalg import Vectors

In [4]:
# dataviz
import seaborn as sns
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

/opt/conda/anaconda/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


<hr />
<hr />
<hr />

In [3]:
training_data = spark.read.parquet('gs://ai-covid19-datalake/trusted/experiment_map/datasets/strat_samp_pcr_agegrp_cols_set_1_0.parquet').persist(StorageLevel.MEMORY_ONLY)
training_data.limit(2).toPandas()

,NU_NOTIFIC,CLASSI_FIN,CRITERIO,EVOLUCAO,AGE_GROUP,DIST_PRI_NOTIFIC_Q,DIST_PRI_INTERNA_Q,DIST_PRI_ENTUTI_Q,DIST_PRI_SAIDUTI_Q,DIST_PRI_EVOLUCA_Q,DIST_PRI_ENCERRA_Q,SYMP_GROUP1,SYMP_GROUP2,SYMP_GROUP3,SYMP_GROUP4,RF_GROUP1,RF_GROUP2,RF_GROUP3,RF_GROUP4,SUPORT_VEN,UTI,HOSPITAL,DIST_PRI_RAIOX_Q,DIST_PRI_COLETA_Q,DIST_PRI_TOMO_Q,DIST_PRI_IF_Q,DIST_PRI_TRA_Q,DIST_PRI_PCR_Q,DIST_PRI_SOR_Q,GMR_TRANSIT_STATIONS_1WEEK_BEFORE_Q,GMR_RETAIL_AND_RECREATION_1WEEK_BEFORE_Q,GMR_RESIDENTIAL_PERCENT_1WEEK_BEFORE_Q,GMR_WORKPLACES_PERCENT_1WEEK_BEFORE_Q,GMR_TRANSIT_STATIONS_2WEEKS_Q,GMR_RETAIL_AND_RECREATION_2WEEKS_Q,GMR_RESIDENTIAL_PERCENT_2WEEKS_Q,GMR_WORKPLACES_PERCENT_2WEEKS_Q,INMET_RELATIVE_AIR_HUMIDITY_1WEEK_BEFORE_Q,INMET_RELATIVE_AIR_HUMIDITY_2WEEKS_BEFORE_Q
0,554,0.0,1.0,1.0,3.0,4.0,3.0,6.0,6.0,3.0,4.0,4.0,3.0,1.0,9.0,1.0,1.0,1.0,1.0,3.0,2.0,1.0,6.0,4.0,6.0,6.0,6.0,3.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0
1,920,0.0,1.0,1.0,1.0,2.0,2.0,6.0,6.0,5.0,5.0,8.0,8.0,1.0,9.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,2.0,3.0,6.0,6.0,6.0,2.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,3.0,3.0


In [4]:
super_srag = spark.read.parquet('gs://ai-covid19-datalake/standard/super-srag/super_srag_v2.parquet')
super_srag.limit(2).toPandas()

,NU_NOTIFIC,CS_SEXO,DT_NASC,AGE_AT_NOTIF,AGE_GROUP,CS_GESTANT,CS_RACA,CS_ETINIA,CS_ESCOL_N,SG_UF,CO_MUN_RES,SEM_PRI,SEM_NOT,DIST_PRI_NOTIFIC,DT_SIN_PRI,SG_UF_NOT,CO_MUN_NOT,SURTO_SG,NOSOCOMIAL,AVE_SUINO,VACINA,HOSPITAL,DIST_PRI_INTERNA,SUPORT_VEN,UTI,DIST_PRI_ENTUTI,CLASSI_OUT,CRITERIO,EVOLUCAO,CLASSI_FIN,SYMP_GROUP1,SYMP_GROUP2,SYMP_GROUP3,SYMP_GROUP4,OUTRO_SIN,OUTRO_DES,RF_GROUP1,RF_GROUP2,RF_GROUP3,RF_GROUP4,OBES_IMC,OUT_MORBI,MORB_DESC,RAIOX_RES,DIST_PRI_RAIOX,TOMO_RES,DIST_PRI_TOMO,AMOSTRA,TP_AMOSTRA,DT_COLETA,...,GMR_PARKS_PERCENT_AVG,GMR_TRANSIT_STATIONS_1WEEK_BEFORE_AVG,GMR_GROCERY_AND_PHARMACY_1WEEK_BEFORE_AVG,GMR_RETAIL_AND_RECREATION_1WEEK_BEFORE_AVG,GMR_WORKPLACES_PERCENT_1WEEK_BEFORE_AVG,GMR_RESIDENTIAL_PERCENT_1WEEK_BEFORE_AVG,GMR_PARKS_PERCENT_1WEEK_BEFORE_AVG,GMR_TRANSIT_STATIONS_2WEEKS_AVG,GMR_GROCERY_AND_PHARMACY_2WEEKS_AVG,GMR_RETAIL_AND_RECREATION_2WEEKS_AVG,GMR_WORKPLACES_PERCENT_2WEEKS_AVG,GMR_RESIDENTIAL_PERCENT_2WEEKS_AVG,GMR_PARKS_PERCENT_2WEEKS_AVG,INMET_TEMP_C_AVG,INMET_RELATIVE_AIR_HUMIDITY_AVG,INMET_DAILY_PRECIPT_AVG,INMET_TEMP_C_1WEEK_BEFORE_AVG,INMET_RELATIVE_AIR_HUMIDITY_1WEEK_BEFORE_AVG,INMET_DAILY_PRECIPT_1WEEK_BEFORE_AVG,INMET_TEMP_C_2WEEKS_BEFORE_AVG,INMET_RELATIVE_AIR_HUMIDITY_2WEEKS_BEFORE_AVG,INMET_DAILY_PRECIPT_2WEEKS_BEFORE_AVG,GMR_TRANSIT_STATIONS_Q,GMR_GROCERY_AND_PHARMACY_Q,GMR_RETAIL_AND_RECREATION_Q,GMR_WORKPLACES_PERCENT_Q,GMR_RESIDENTIAL_PERCENT_Q,GMR_PARKS_PERCENT_Q,GMR_TRANSIT_STATIONS_1WEEK_BEFORE_Q,GMR_GROCERY_AND_PHARMACY_1WEEK_BEFORE_Q,GMR_RETAIL_AND_RECREATION_1WEEK_BEFORE_Q,GMR_WORKPLACES_PERCENT_1WEEK_BEFORE_Q,GMR_RESIDENTIAL_PERCENT_1WEEK_BEFORE_Q,GMR_PARKS_PERCENT_1WEEK_BEFORE_Q,GMR_TRANSIT_STATIONS_2WEEKS_Q,GMR_GROCERY_AND_PHARMACY_2WEEKS_Q,GMR_RETAIL_AND_RECREATION_2WEEKS_Q,GMR_WORKPLACES_PERCENT_2WEEKS_Q,GMR_RESIDENTIAL_PERCENT_2WEEKS_Q,GMR_PARKS_PERCENT_2WEEKS_Q,INMET_TEMP_C_Q,INMET_RELATIVE_AIR_HUMIDITY_Q,INMET_DAILY_PRECIPT_Q,INMET_TEMP_C_1WEEK_BEFORE_Q,INMET_RELATIVE_AIR_HUMIDITY_1WEEK_BEFORE_Q,INMET_DAILY_PRECIPT_1WEEK_BEFORE_Q,INMET_TEMP_C_2WEEKS_BEFORE_Q,INMET_RELATIVE_AIR_HUMIDITY_2WEEKS_BEFORE_Q,INMET_DAILY_PRECIPT_2WEEKS_BEFORE_Q,ANO
0,68719768275,M,1967-01-25,54,7,6,4,None,NaN,BA,290240,19,20,7,2021-05-15,BA,291800,None,NaN,NaN,None,1,NaN,NaN,NaN,None,None,None,None,None,9,3,2,1,NaN,None,1,1,1,1,None,None,None,None,None,None,None,2,NaN,None,...,-44.62533,-25.097436,34.628571,-20.930314,0.773764,6.81768,-44.472296,-31.233161,17.257009,-30.477193,-4.605239,8.130556,-46.526854,22.265185,71.299468,1.115789,23.258355,74.984254,2.16,24.092008,74.733094,1.653012,3,5,3,4,2,2,4,5,4,5,2,2,3,4,3,3,3,1,2,3,2,3,4,3,4,4,2,2021
1,68719899483,M,1959-10-20,61,8,6,4,None,9.0,BA,292050,19,20,8,2021-05-11,BA,292050,None,2.0,2.0,None,1,8.0,3.0,2.0,None,None,None,None,None,3,1,3,1,1.0,ASTENIA E INAPETENCIA,1,1,1,1,None,None,None,None,None,6,None,1,1.0,2021-05-17,...,-44.62533,-25.097436,34.628571,-20.930314,0.773764,6.81768,-44.472296,-31.233161,17.257009,-30.477193,-4.605239,8.130556,-46.526854,22.265185,71.299468,1.115789,23.258355,74.984254,2.16,24.092008,74.733094,1.653012,3,5,3,4,2,2,4,5,4,5,2,2,3,4,3,3,3,1,2,3,2,3,4,3,4,4,2,2021


In [5]:
super_srag_to_reclass = super_srag.filter((F.col('CLASSI_FIN').isNotNull()) & (F.col('CLASSI_FIN') == 4))
super_srag_to_reclass.count()

882348

<hr />
<hr />
<hr />

In [6]:
cols_set_1 = ['NU_NOTIFIC', 'CLASSI_FIN', 'CRITERIO', 'EVOLUCAO', 
              'AGE_GROUP', 'DIST_PRI_NOTIFIC_Q',
              'DIST_PRI_INTERNA_Q', 'DIST_PRI_ENTUTI_Q', 'DIST_PRI_SAIDUTI_Q', 'DIST_PRI_EVOLUCA_Q', 'DIST_PRI_ENCERRA_Q',
              'SYMP_GROUP1', 'SYMP_GROUP2', 'SYMP_GROUP3', 'SYMP_GROUP4',
              'RF_GROUP1', 'RF_GROUP2', 'RF_GROUP3', 'RF_GROUP4',
              'SUPORT_VEN', 'UTI', 'HOSPITAL',
              'DIST_PRI_RAIOX_Q', 'DIST_PRI_COLETA_Q', 'DIST_PRI_TOMO_Q', 'DIST_PRI_IF_Q', 'DIST_PRI_TRA_Q', 'DIST_PRI_PCR_Q', 'DIST_PRI_SOR_Q',
              'GMR_TRANSIT_STATIONS_1WEEK_BEFORE_Q', 'GMR_RETAIL_AND_RECREATION_1WEEK_BEFORE_Q', 'GMR_RESIDENTIAL_PERCENT_1WEEK_BEFORE_Q', 'GMR_WORKPLACES_PERCENT_1WEEK_BEFORE_Q',
              'GMR_TRANSIT_STATIONS_2WEEKS_Q', 'GMR_RETAIL_AND_RECREATION_2WEEKS_Q', 'GMR_RESIDENTIAL_PERCENT_2WEEKS_Q', 'GMR_WORKPLACES_PERCENT_2WEEKS_Q',
              'INMET_RELATIVE_AIR_HUMIDITY_1WEEK_BEFORE_Q', 'INMET_RELATIVE_AIR_HUMIDITY_2WEEKS_BEFORE_Q']

id_cols = ['NU_NOTIFIC', 'CLASSI_FIN']

<hr />
<hr />
<hr />

# Training model

In [7]:
def dropDupeDfCols(df):
    newcols = []
    dupcols = []

    for i in range(len(df.columns)):
        if df.columns[i] not in newcols:
            newcols.append(df.columns[i])
        else:
            dupcols.append(i)

    df = df.toDF(*[str(i) for i in range(len(df.columns))])
    for dupcol in dupcols:
        df = df.drop(str(dupcol))

    return df.toDF(*newcols)

def format_data(df, reclass=False):

    nominal_cols = ['CS_SEXO', 'SG_UF']
    
    df_ = df.na.fill('9999')
    df_ = df_.na.fill(9999)
    
    for col in df.columns:
        if col in nominal_cols:
            df_ = df_.withColumn('CS_SEXO', F.when(F.col('CS_SEXO') == 'F', '2').otherwise('1'))
        if col not in ['NU_NOTIFIC']:
            df_ = df_.withColumn(col, F.col(col).cast('float'))

#     df_ = df_.dropna()
    if reclass == True:
        df_ = df_.withColumn('CLASSI_FIN', F.lit(0.0))
        df_ = df_.filter(F.col('RF_GROUP1') != 8.0)
    
    return dropDupeDfCols(df_)

In [8]:
super_srag_to_reclass = format_data(super_srag_to_reclass.select(cols_set_1), reclass=True)
super_srag_to_reclass.limit(2).toPandas()

,NU_NOTIFIC,CLASSI_FIN,CRITERIO,EVOLUCAO,AGE_GROUP,DIST_PRI_NOTIFIC_Q,DIST_PRI_INTERNA_Q,DIST_PRI_ENTUTI_Q,DIST_PRI_SAIDUTI_Q,DIST_PRI_EVOLUCA_Q,DIST_PRI_ENCERRA_Q,SYMP_GROUP1,SYMP_GROUP2,SYMP_GROUP3,SYMP_GROUP4,RF_GROUP1,RF_GROUP2,RF_GROUP3,RF_GROUP4,SUPORT_VEN,UTI,HOSPITAL,DIST_PRI_RAIOX_Q,DIST_PRI_COLETA_Q,DIST_PRI_TOMO_Q,DIST_PRI_IF_Q,DIST_PRI_TRA_Q,DIST_PRI_PCR_Q,DIST_PRI_SOR_Q,GMR_TRANSIT_STATIONS_1WEEK_BEFORE_Q,GMR_RETAIL_AND_RECREATION_1WEEK_BEFORE_Q,GMR_RESIDENTIAL_PERCENT_1WEEK_BEFORE_Q,GMR_WORKPLACES_PERCENT_1WEEK_BEFORE_Q,GMR_TRANSIT_STATIONS_2WEEKS_Q,GMR_RETAIL_AND_RECREATION_2WEEKS_Q,GMR_RESIDENTIAL_PERCENT_2WEEKS_Q,GMR_WORKPLACES_PERCENT_2WEEKS_Q,INMET_RELATIVE_AIR_HUMIDITY_1WEEK_BEFORE_Q,INMET_RELATIVE_AIR_HUMIDITY_2WEEKS_BEFORE_Q
0,77309844813,0.0,9999.0,2.0,9.0,2.0,2.0,6.0,6.0,1.0,1.0,1.0,5.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,6.0,6.0,2.0,6.0,6.0,6.0,6.0,4.0,4.0,2.0,5.0,3.0,3.0,3.0,3.0,4.0,4.0
1,85899754416,0.0,1.0,1.0,8.0,3.0,3.0,6.0,6.0,1.0,1.0,1.0,8.0,1.0,1.0,9.0,9.0,9.0,9.0,2.0,2.0,1.0,6.0,3.0,6.0,6.0,6.0,6.0,6.0,4.0,4.0,2.0,5.0,3.0,3.0,3.0,3.0,4.0,4.0


In [9]:
training_data = format_data(training_data)

In [10]:
training_data.select('CLASSI_FIN').groupBy('CLASSI_FIN').count().show()

+----------+-----+
|CLASSI_FIN|count|
+----------+-----+
|       1.0|76847|
|       0.0|44537|
+----------+-----+



In [11]:
# Index labels, adding metadata to the label column.
# Fit on whole dataset to include all labels in index.
labelIndexer = StringIndexer(inputCol="CLASSI_FIN", outputCol="indexedLabel").fit(training_data)

In [12]:
input_cols = [x for x in training_data.columns if x not in id_cols]
assembler = VectorAssembler(inputCols = input_cols, outputCol= 'features')
training_data = assembler.transform(training_data)

In [13]:
# Automatically identify categorical features, and index them.
# Set maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=20).fit(training_data)

In [14]:
# Train a RandomForest model.
gbt = GBTClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", maxIter=100, maxDepth=7, maxBins=32)

In [15]:
# Convert indexed labels back to original labels.
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel",
                               labels=labelIndexer.labels)

In [16]:
# Chain indexers and forest in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, gbt, labelConverter])

In [17]:
# Train model.  This also runs the indexers.
model = pipeline.fit(training_data)

<hr />
<hr />
<hr />

# Reclassificando

In [18]:
super_srag_to_reclass = format_data(super_srag_to_reclass.select(cols_set_1), reclass=True)
super_srag_to_reclass.limit(2).toPandas()

,NU_NOTIFIC,CLASSI_FIN,CRITERIO,EVOLUCAO,AGE_GROUP,DIST_PRI_NOTIFIC_Q,DIST_PRI_INTERNA_Q,DIST_PRI_ENTUTI_Q,DIST_PRI_SAIDUTI_Q,DIST_PRI_EVOLUCA_Q,DIST_PRI_ENCERRA_Q,SYMP_GROUP1,SYMP_GROUP2,SYMP_GROUP3,SYMP_GROUP4,RF_GROUP1,RF_GROUP2,RF_GROUP3,RF_GROUP4,SUPORT_VEN,UTI,HOSPITAL,DIST_PRI_RAIOX_Q,DIST_PRI_COLETA_Q,DIST_PRI_TOMO_Q,DIST_PRI_IF_Q,DIST_PRI_TRA_Q,DIST_PRI_PCR_Q,DIST_PRI_SOR_Q,GMR_TRANSIT_STATIONS_1WEEK_BEFORE_Q,GMR_RETAIL_AND_RECREATION_1WEEK_BEFORE_Q,GMR_RESIDENTIAL_PERCENT_1WEEK_BEFORE_Q,GMR_WORKPLACES_PERCENT_1WEEK_BEFORE_Q,GMR_TRANSIT_STATIONS_2WEEKS_Q,GMR_RETAIL_AND_RECREATION_2WEEKS_Q,GMR_RESIDENTIAL_PERCENT_2WEEKS_Q,GMR_WORKPLACES_PERCENT_2WEEKS_Q,INMET_RELATIVE_AIR_HUMIDITY_1WEEK_BEFORE_Q,INMET_RELATIVE_AIR_HUMIDITY_2WEEKS_BEFORE_Q
0,77309844813,0.0,9999.0,2.0,9.0,2.0,2.0,6.0,6.0,1.0,1.0,1.0,5.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,6.0,6.0,2.0,6.0,6.0,6.0,6.0,4.0,4.0,2.0,5.0,3.0,3.0,3.0,3.0,4.0,4.0
1,85899754416,0.0,1.0,1.0,8.0,3.0,3.0,6.0,6.0,1.0,1.0,1.0,8.0,1.0,1.0,9.0,9.0,9.0,9.0,2.0,2.0,1.0,6.0,3.0,6.0,6.0,6.0,6.0,6.0,4.0,4.0,2.0,5.0,3.0,3.0,3.0,3.0,4.0,4.0


In [19]:
super_srag_to_reclass.select('CLASSI_FIN').groupBy('CLASSI_FIN').count().show()

+----------+------+
|CLASSI_FIN| count|
+----------+------+
|       0.0|882315|
+----------+------+



In [20]:
input_cols = [x for x in super_srag_to_reclass.columns if x not in id_cols]
assembler = VectorAssembler(inputCols = input_cols, outputCol= 'features') #handleInvalid='skip'
reclas_ss = assembler.transform(super_srag_to_reclass)

In [21]:
# Automatically identify categorical features, and index them.
# Set maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=20).fit(reclas_ss.select(input_cols + ['features', 'NU_NOTIFIC'])) #handleInvalid='skip'

In [22]:
reclas_ss_pred = model.transform(reclas_ss)

In [23]:
reclas_ss_pred.limit(3).toPandas()

,NU_NOTIFIC,CLASSI_FIN,CRITERIO,EVOLUCAO,AGE_GROUP,DIST_PRI_NOTIFIC_Q,DIST_PRI_INTERNA_Q,DIST_PRI_ENTUTI_Q,DIST_PRI_SAIDUTI_Q,DIST_PRI_EVOLUCA_Q,DIST_PRI_ENCERRA_Q,SYMP_GROUP1,SYMP_GROUP2,SYMP_GROUP3,SYMP_GROUP4,RF_GROUP1,RF_GROUP2,RF_GROUP3,RF_GROUP4,SUPORT_VEN,UTI,HOSPITAL,DIST_PRI_RAIOX_Q,DIST_PRI_COLETA_Q,DIST_PRI_TOMO_Q,DIST_PRI_IF_Q,DIST_PRI_TRA_Q,DIST_PRI_PCR_Q,DIST_PRI_SOR_Q,GMR_TRANSIT_STATIONS_1WEEK_BEFORE_Q,GMR_RETAIL_AND_RECREATION_1WEEK_BEFORE_Q,GMR_RESIDENTIAL_PERCENT_1WEEK_BEFORE_Q,GMR_WORKPLACES_PERCENT_1WEEK_BEFORE_Q,GMR_TRANSIT_STATIONS_2WEEKS_Q,GMR_RETAIL_AND_RECREATION_2WEEKS_Q,GMR_RESIDENTIAL_PERCENT_2WEEKS_Q,GMR_WORKPLACES_PERCENT_2WEEKS_Q,INMET_RELATIVE_AIR_HUMIDITY_1WEEK_BEFORE_Q,INMET_RELATIVE_AIR_HUMIDITY_2WEEKS_BEFORE_Q,features,indexedLabel,indexedFeatures,rawPrediction,probability,prediction,predictedLabel
0,77309844813,0.0,9999.0,2.0,9.0,2.0,2.0,6.0,6.0,1.0,1.0,1.0,5.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,6.0,6.0,2.0,6.0,6.0,6.0,6.0,4.0,4.0,2.0,5.0,3.0,3.0,3.0,3.0,4.0,4.0,"[9999.0, 2.0, 9.0, 2.0, 2.0, 6.0, 6.0, 1.0, 1....",1.0,"[4.0, 1.0, 8.0, 1.0, 1.0, 5.0, 5.0, 0.0, 0.0, ...","[-1.8874597667344128, 1.8874597667344128]","[0.022424541258921682, 0.9775754587410783]",1.0,0.0
1,85899754416,0.0,1.0,1.0,8.0,3.0,3.0,6.0,6.0,1.0,1.0,1.0,8.0,1.0,1.0,9.0,9.0,9.0,9.0,2.0,2.0,1.0,6.0,3.0,6.0,6.0,6.0,6.0,6.0,4.0,4.0,2.0,5.0,3.0,3.0,3.0,3.0,4.0,4.0,"[1.0, 1.0, 8.0, 3.0, 3.0, 6.0, 6.0, 1.0, 1.0, ...",1.0,"[0.0, 0.0, 7.0, 2.0, 2.0, 5.0, 5.0, 0.0, 0.0, ...","[-2.213018281798712, 2.213018281798712]","[0.011820412225632683, 0.9881795877743673]",1.0,0.0
2,77309663864,0.0,1.0,9999.0,8.0,2.0,3.0,6.0,6.0,6.0,1.0,3.0,1.0,1.0,1.0,3.0,1.0,5.0,1.0,9999.0,2.0,1.0,6.0,3.0,6.0,6.0,3.0,6.0,6.0,4.0,4.0,2.0,5.0,3.0,3.0,3.0,3.0,4.0,4.0,"[1.0, 9999.0, 8.0, 2.0, 3.0, 6.0, 6.0, 6.0, 1....",1.0,"[0.0, 4.0, 7.0, 1.0, 2.0, 5.0, 5.0, 5.0, 0.0, ...","[-2.1532151538209634, 2.1532151538209634]","[0.013302253413081363, 0.9866977465869187]",1.0,0.0


<hr />
<hr />
<hr />

In [42]:
super_srag.select('CLASSI_FIN').printSchema()

root
 |-- CLASSI_FIN: integer (nullable = true)



<hr />
<hr />
<hr />

# Explorando

In [31]:
reclas_ss_pred = reclas_ss_pred.withColumnRenamed('predictedLabel', 'CLASSI_FIN_pred')

In [32]:
reclas_ss_pred.select('CLASSI_FIN_pred').groupBy('CLASSI_FIN_pred').count().show()

+---------------+------+
|CLASSI_FIN_pred| count|
+---------------+------+
|            1.0|528063|
|            0.0|354252|
+---------------+------+



In [45]:
super_srag_ = super_srag.filter((F.col('CLASSI_FIN') != 4) & (F.col('CLASSI_FIN').isNotNull()))
super_srag_ = super_srag_.withColumn('CLASSI_FIN_source', F.lit('original'))

In [35]:
reclas_ss_pred = reclas_ss_pred.withColumn('CLASSI_FIN_source', F.lit('predicted'))
reclas_ss_pred = reclas_ss_pred.drop('CLASSI_FIN').withColumnRenamed('CLASSI_FIN_pred', 'CLASSI_FIN')
reclas_ss_pred = reclas_ss_pred.withColumn('CLASSI_FIN', F.when(F.col('CLASSI_FIN') == 1, 5).otherwise(F.lit(4)))
reclas_ss_pred = reclas_ss_pred.withColumn('CLASSI_FIN', F.col('CLASSI_FIN').cast('integer'))

In [36]:
reclas_ss_pred.select('CLASSI_FIN').groupBy('CLASSI_FIN').count().show()

+----------+------+
|CLASSI_FIN| count|
+----------+------+
|         5|528063|
|         4|354252|
+----------+------+



In [46]:
super_srag_.select('CLASSI_FIN', 'CLASSI_FIN_source').groupBy(['CLASSI_FIN', 'CLASSI_FIN_source']).count().show()

+----------+-----------------+-------+
|CLASSI_FIN|CLASSI_FIN_source|  count|
+----------+-----------------+-------+
|         3|         original|   9194|
|         2|         original|  33269|
|         5|         original|2454142|
|         1|         original|  11269|
+----------+-----------------+-------+



In [44]:
super_srag_.select('CLASSI_FIN').printSchema()

root
 |-- CLASSI_FIN: integer (nullable = true)



In [38]:
union_cols = ['NU_NOTIFIC', 'CLASSI_FIN', 'CLASSI_FIN_source']

In [47]:
super_srag_ = super_srag_.select(union_cols)
reclas_ss_pred = reclas_ss_pred.select(union_cols)
super_srag_ = super_srag_.union(reclas_ss_pred)

In [49]:
super_srag_.select(['CLASSI_FIN', 'CLASSI_FIN_source']).groupBy(['CLASSI_FIN', 'CLASSI_FIN_source']).count().show()

+----------+-----------------+-------+
|CLASSI_FIN|CLASSI_FIN_source|  count|
+----------+-----------------+-------+
|         5|        predicted| 528063|
|         3|         original|   9194|
|         2|         original|  33269|
|         4|        predicted| 354252|
|         5|         original|2454142|
|         1|         original|  11269|
+----------+-----------------+-------+



In [50]:
super_srag_.limit(2).toPandas()

,NU_NOTIFIC,CLASSI_FIN,CLASSI_FIN_source
0,77309859740,5,original
1,68719688318,5,original


In [51]:
super_srag = super_srag.drop('CLASSI_FIN')

In [52]:
super_srag = super_srag.join(super_srag_, 'NU_NOTIFIC', 'left')

In [54]:
super_srag = super_srag.filter(F.col('CLASSI_FIN').isNotNull())

In [55]:
super_srag.select(['CLASSI_FIN', 'CLASSI_FIN_source']).groupBy(['CLASSI_FIN', 'CLASSI_FIN_source']).count().show()

+----------+-----------------+-------+
|CLASSI_FIN|CLASSI_FIN_source|  count|
+----------+-----------------+-------+
|         5|        predicted| 528063|
|         3|         original|   9194|
|         2|         original|  33269|
|         4|        predicted| 354252|
|         5|         original|2454142|
|         1|         original|  11269|
+----------+-----------------+-------+



In [57]:
super_srag.write.parquet('gs://ai-covid19-datalake/trusted/reclassified_srag_16-12-2021_gbt_pcr_stratsamp.parquet', mode='overwrite')

<hr />
<hr />
<hr />

In [5]:
# reading from checkpoint
super_srag = spark.read.parquet('gs://ai-covid19-datalake/trusted/reclassified_srag_16-12-2021_gbt_pcr_stratsamp.parquet').persist(StorageLevel.MEMORY_ONLY)

# Visualizing amount of predicted cases accross epidemiological weeks

In [6]:
def make_attr(classi_fin, classi_fin_source):
    if classi_fin == 5:
        classi_fin = 'covid'
    else:
        classi_fin = 'not-covid'
    if classi_fin_source == 'predicted':
        suffix = 'pred'
    else: 
        suffix = 'ori'
    return str(classi_fin) + '-' + str(suffix)
udf_make_attr = F.udf(make_attr, StringType())    

In [7]:
super_srag.select('ANO').printSchema()

root
 |-- ANO: integer (nullable = true)



In [8]:
ss_weeks = super_srag.groupBy(['ANO', 'epi_week_year', 'CLASSI_FIN', 'CLASSI_FIN_source']).agg({'CLASSI_FIN_source': 'count', 'CLASSI_FIN': 'count'}).orderBy(['ANO', 'epi_week_year']).select(['ANO', 'epi_week_year', 'CLASSI_FIN_source', 'CLASSI_FIN', 'count(CLASSI_FIN)']).withColumn('attr', udf_make_attr(F.col('CLASSI_FIN'), F.col('CLASSI_FIN_source')))

In [9]:
# filtering ss_weeks
years = [2020, 2021]
ss_weeks = ss_weeks.filter(F.col('ANO').isin(years))

In [10]:
ss_weeks.limit(10).toPandas()

,ANO,epi_week_year,CLASSI_FIN_source,CLASSI_FIN,count(CLASSI_FIN),attr
0,2020,01-2020,predicted,5,49,covid-pred
1,2020,01-2020,original,2,39,not-covid-ori
2,2020,01-2020,original,3,10,not-covid-ori
3,2020,01-2020,predicted,4,165,not-covid-pred
4,2020,01-2020,original,1,20,not-covid-ori
5,2020,02-2020,original,1,58,not-covid-ori
6,2020,02-2020,original,3,3,not-covid-ori
7,2020,02-2020,predicted,4,245,not-covid-pred
8,2020,02-2020,original,2,44,not-covid-ori
9,2020,02-2020,predicted,5,84,covid-pred


In [11]:
# TODO: generate and organize all epidemiological weeks automatically

# # putting all epi_week_year tags in a list to create the bars on plot
# weeks_cols = ss_weeks.select(['ANO', 'epi_week_year']).distinct().select('epi_week_year').rdd.flatMap(lambda x: x).collect()

# # for some reason, there is a duplicate for epi_week_year '01-2020', we have to pop it
# weeks_cols.pop(2)

weeks_cols = ['01-2020', '02-2020', '03-2020', '04-2020', '05-2020', '06-2020', '07-2020', '08-2020', '09-2020', '10-2020', 
 '11-2020', '12-2020', '13-2020', '14-2020', '15-2020', '16-2020', '17-2020', '18-2020', '19-2020', '20-2020',
 '21-2020', '22-2020', '23-2020', '24-2020', '25-2020', '26-2020', '27-2020', '28-2020', '29-2020', '30-2020',
 '31-2020', '32-2020', '33-2020', '34-2020', '35-2020', '36-2020', '37-2020', '38-2020', '39-2020', '40-2020', 
 '41-2020', '42-2020', '43-2020', '44-2020', '45-2020', '46-2020', '47-2020', '48-2020', '49-2020', '50-2020',
 '51-2020', '52-2020', '53-2020', 
 '01-2021', '02-2021', '03-2021', '04-2021', '05-2021', '06-2021','07-2021', '08-2021', '09-2021', '10-2021',
 '11-2021', '12-2021', '13-2021', '14-2021', '15-2021', '16-2021', '17-2021', '18-2021', '19-2021', '20-2021',
 '21-2021', '22-2021', '23-2021', '24-2021', '25-2021', '26-2021', '27-2021', '28-2021', '29-2021', '30-2021',
 '31-2021', '32-2021', '33-2021', '34-2021', '35-2021', '36-2021', '37-2021', '38-2021', '39-2021', '40-2021',
 '41-2021', '42-2021', '43-2021', '44-2021', '45-2021', '46-2021']

In [12]:
# creating a empty dataframe to join the counts for each epi_week after
to_plot = spark.createDataFrame(["covid-pred", "covid-ori", "not-covid-pred", "not-covid-ori"], StringType()).toDF("attr")

# attaching the counts of each epi_week on the plot dataframe
for week in weeks_cols:
    week_counts = ss_weeks.filter(F.col('epi_week_year') == week).select(['attr', 'count(CLASSI_FIN)']).groupBy('attr').agg({'count(CLASSI_FIN)': 'sum'}).withColumnRenamed('sum(count(CLASSI_FIN))', week)
    to_plot = to_plot.join(week_counts, 'attr', 'left')

pd_to_plot = to_plot.toPandas()

In [13]:
pd_to_plot

,attr,01-2020,02-2020,03-2020,04-2020,05-2020,06-2020,07-2020,08-2020,09-2020,10-2020,11-2020,12-2020,13-2020,14-2020,15-2020,16-2020,17-2020,18-2020,19-2020,20-2020,21-2020,22-2020,23-2020,24-2020,25-2020,26-2020,27-2020,28-2020,29-2020,30-2020,31-2020,32-2020,33-2020,34-2020,35-2020,36-2020,37-2020,38-2020,39-2020,40-2020,41-2020,42-2020,43-2020,44-2020,45-2020,46-2020,47-2020,48-2020,49-2020,50-2020,51-2020,52-2020,53-2020,01-2021,02-2021,03-2021,04-2021,05-2021,06-2021,07-2021,08-2021,09-2021,10-2021,11-2021,12-2021,13-2021,14-2021,15-2021,16-2021,17-2021,18-2021,19-2021,20-2021,21-2021,22-2021,23-2021,24-2021,25-2021,26-2021,27-2021,28-2021,29-2021,30-2021,31-2021,32-2021,33-2021,34-2021,35-2021,36-2021,37-2021,38-2021,39-2021,40-2021,41-2021,42-2021,43-2021,44-2021,45-2021,46-2021
0,not-covid-ori,69.0,105.0,110.0,97.0,126.0,131.0,161.0,213,341,374,652,750,524,312,228,163,114,108,119,127,81,87,112,125,111,125,152,117,164,167,169,188,214,217,215,195,136,159,149,152,167,161,180,154,195,202,185,183,208,186,183,198,204,549,546,601,561,662,742,839,905,1092,1076,1070,913,869,788,741,707,679,695,631,537,466,473,476,453,470,533,578,574,544,451,541,506,466,435,449,421,418,498,476,406,429,447,422,415,291,134
1,covid-pred,49.0,84.0,74.0,96.0,98.0,80.0,84.0,71,122,170,569,2033,3612,5629,6996,7390,8231,8627,8437,9050,8646,8414,9518,8715,9097,8330,9201,9300,8677,8685,8818,8487,8576,7577,7403,7585,7101,6756,6336,6704,6171,5476,5266,5286,5615,5456,5623,4529,5166,4463,4034,4759,4948,10600,10929,10132,9289,10200,9434,8933,7521,9078,8388,8620,7387,7899,7961,7330,6796,6975,6182,5688,4431,4101,4008,3736,3875,4114,4164,4230,3849,3696,3348,3997,3758,2933,2426,2525,2324,2184,2219,1881,1740,1795,1651,1295,1121,742,252
2,not-covid-pred,165.0,245.0,261.0,225.0,247.0,269.0,356.0,522,579,1073,2558,5970,5032,2859,1706,2152,2752,3109,2680,2811,2470,2078,2425,2423,2636,2432,2738,2892,3057,3230,3048,2888,3423,3136,3092,2881,2566,2502,2391,3108,2833,2608,2815,1967,2664,2754,2654,3066,3149,3319,3303,2504,2588,5875,5665,5592,5404,5987,5924,6791,6473,8167,7978,7766,6239,5673,5909,5155,5288,5220,5055,5145,4246,3657,3536,3235,3071,3032,3173,3333,3304,3232,3122,2965,3147,3725,3380,3122,3112,2955,3023,2961,2708,2902,2737,2381,2464,1687,696
3,covid-ori,NaN,NaN,NaN,NaN,NaN,NaN,NaN,57,75,265,1140,3629,5368,6948,9876,14047,18245,21331,21664,23434,21000,19211,21958,21527,23279,20684,23715,23162,21068,20151,18645,17007,17317,15870,14219,14388,12843,11541,11257,10820,10279,9370,9650,9781,13080,15486,18563,19039,21775,19994,18482,20508,23627,49711,47014,42550,39748,49400,56110,74603,82249,116952,118713,116447,92064,85591,79176,66422,64002,64025,58978,55497,45334,42781,41578,35132,29508,24933,22226,19889,16148,15003,12663,13522,11185,10163,8363,7919,7187,6636,6136,5379,4058,3768,3213,2667,2169,1376,557


In [20]:
# plotting cases (original and predicted) by epidemiological weeks
pd_to_plot.set_index('attr').T.plot(kind='bar', stacked=True,
          colormap=ListedColormap(sns.color_palette()), width=0.5,
          figsize=(200,70))

plt.xticks(fontsize=80, rotation=45)
plt.yticks(fontsize=120)
plt.legend(fontsize=150)
axes = plt.gca()
axes.set_ylim([0,145000])
plt.grid(color='gray', linestyle='-', linewidth=2)
plt.title(label="Final classification of cases after prediction - from jan/2020 to dez/2021 - BR", loc='center', fontsize=200)
plt.show()

<hr />
<hr />
<hr />

# Visualizing amount of predicted outcomes accross epidemiological weeks

In [29]:
def make_attr2(classi_fin, classi_fin_source, evolucao):
    if classi_fin == 5:
        classi_fin = 'covid'
    else:
        classi_fin = 'not-covid'
    if classi_fin_source == 'predicted':
        suffix = 'pred'
    else: 
        suffix = 'ori'
    
    if evolucao == '1':
        ev = 'recovered'
    elif (evolucao == '2') or (evolucao == '3'):
        ev = 'deaths'
    else:
        ev = 'ignored'
    return str(classi_fin) + '-' + suffix + '-' + ev
udf_make_attr2 = F.udf(make_attr2, StringType())    

In [30]:
ss_weeks = super_srag.groupBy(['ANO', 'epi_week_year', 'EVOLUCAO', 'CLASSI_FIN', 'CLASSI_FIN_source']).agg({'CLASSI_FIN_source': 'count', 'CLASSI_FIN': 'count'}).orderBy(['ANO', 'epi_week_year']).select(['ANO', 'epi_week_year', 'EVOLUCAO', 'CLASSI_FIN_source', 'CLASSI_FIN', 'count(CLASSI_FIN)']).withColumn('attr', udf_make_attr2(F.col('CLASSI_FIN'), F.col('CLASSI_FIN_source'), F.col('EVOLUCAO')))

In [31]:
# filtering ss_weeks
years = [2020, 2021]
ss_weeks = ss_weeks.filter(F.col('ANO').isin(years))

In [32]:
ss_weeks.limit(10).toPandas()

,ANO,epi_week_year,EVOLUCAO,CLASSI_FIN_source,CLASSI_FIN,count(CLASSI_FIN),attr
0,2020,01-2020,3,predicted,4,3,not-covid-pred-deaths
1,2020,01-2020,3,predicted,5,1,covid-pred-deaths
2,2020,01-2020,9,original,2,1,not-covid-ori-ignored
3,2020,01-2020,1,original,1,17,not-covid-ori-recovered
4,2020,01-2020,1,predicted,4,134,not-covid-pred-recovered
5,2020,01-2020,None,predicted,5,5,covid-pred-ignored
6,2020,01-2020,2,original,2,1,not-covid-ori-deaths
7,2020,01-2020,2,original,3,6,not-covid-ori-deaths
8,2020,01-2020,None,original,2,3,not-covid-ori-ignored
9,2020,01-2020,2,predicted,5,11,covid-pred-deaths


In [38]:
# TODO: generate and organize all epidemiological weeks automatically

# # putting all epi_week_year tags in a list to create the bars on plot
# weeks_cols = ss_weeks.select(['ANO', 'epi_week_year']).distinct().select('epi_week_year').rdd.flatMap(lambda x: x).collect()

# # for some reason, there is a duplicate for epi_week_year '01-2020', we have to pop it
# weeks_cols.pop(2)

weeks_cols = ['01-2020', '02-2020', '03-2020', '04-2020', '05-2020', '06-2020', '07-2020', '08-2020', '09-2020', '10-2020', 
 '11-2020', '12-2020', '13-2020', '14-2020', '15-2020', '16-2020', '17-2020', '18-2020', '19-2020', '20-2020',
 '21-2020', '22-2020', '23-2020', '24-2020', '25-2020', '26-2020', '27-2020', '28-2020', '29-2020', '30-2020',
 '31-2020', '32-2020', '33-2020', '34-2020', '35-2020', '36-2020', '37-2020', '38-2020', '39-2020', '40-2020', 
 '41-2020', '42-2020', '43-2020', '44-2020', '45-2020', '46-2020', '47-2020', '48-2020', '49-2020', '50-2020',
 '51-2020', '52-2020', '53-2020', 
 '01-2021', '02-2021', '03-2021', '04-2021', '05-2021', '06-2021','07-2021', '08-2021', '09-2021', '10-2021',
 '11-2021', '12-2021', '13-2021', '14-2021', '15-2021', '16-2021', '17-2021', '18-2021', '19-2021', '20-2021',
 '21-2021', '22-2021', '23-2021', '24-2021', '25-2021', '26-2021', '27-2021', '28-2021', '29-2021', '30-2021',
 '31-2021', '32-2021', '33-2021', '34-2021', '35-2021', '36-2021', '37-2021', '38-2021', '39-2021', '40-2021',
 '41-2021', '42-2021', '43-2021', '44-2021', '45-2021', '46-2021']

In [39]:
# creating a empty dataframe to join the counts for each epi_week after
# to_plot = spark.createDataFrame(["covid-pred-recovered", "covid-ori-recovered", "not-covid-pred-recovered", "not-covid-ori-recovered",
#                                  "covid-pred-deaths", "covid-ori-deaths", "not-covid-pred-deaths", "not-covid-ori-deaths",
#                                  "covid-pred-ignored", "covid-ori-ignored", "not-covid-pred-ignored", "not-covid-ori-ignored",
#                                 ], StringType()).toDF("attr")

# to_plot = spark.createDataFrame(["covid-pred-recovered", "covid-ori-recovered",
#                                  "covid-pred-deaths", "covid-ori-deaths",
#                                  "covid-pred-ignored", "covid-ori-ignored"
#                                 ], StringType()).toDF("attr")

to_plot = spark.createDataFrame(["covid-pred-recovered", "covid-ori-recovered",
                                 "covid-pred-deaths", "covid-ori-deaths"
                                ], StringType()).toDF("attr")


# attaching the counts of each epi_week on the plot dataframe
for week in weeks_cols:
    week_counts = ss_weeks.filter(F.col('epi_week_year') == week).select(['attr', 'count(CLASSI_FIN)']).groupBy('attr').agg({'count(CLASSI_FIN)': 'sum'}).withColumnRenamed('sum(count(CLASSI_FIN))', week)
    to_plot = to_plot.join(week_counts, 'attr', 'left')

pd_to_plot = to_plot.toPandas()

In [40]:
pd_to_plot

,attr,01-2020,02-2020,03-2020,04-2020,05-2020,06-2020,07-2020,08-2020,09-2020,10-2020,11-2020,12-2020,13-2020,14-2020,15-2020,16-2020,17-2020,18-2020,19-2020,20-2020,21-2020,22-2020,23-2020,24-2020,25-2020,26-2020,27-2020,28-2020,29-2020,30-2020,31-2020,32-2020,33-2020,34-2020,35-2020,36-2020,37-2020,38-2020,39-2020,40-2020,41-2020,42-2020,43-2020,44-2020,45-2020,46-2020,47-2020,48-2020,49-2020,50-2020,51-2020,52-2020,53-2020,01-2021,02-2021,03-2021,04-2021,05-2021,06-2021,07-2021,08-2021,09-2021,10-2021,11-2021,12-2021,13-2021,14-2021,15-2021,16-2021,17-2021,18-2021,19-2021,20-2021,21-2021,22-2021,23-2021,24-2021,25-2021,26-2021,27-2021,28-2021,29-2021,30-2021,31-2021,32-2021,33-2021,34-2021,35-2021,36-2021,37-2021,38-2021,39-2021,40-2021,41-2021,42-2021,43-2021,44-2021,45-2021,46-2021
0,covid-pred-deaths,12.0,24.0,18.0,23.0,33.0,27.0,23.0,21,35,39,148,531,1124,1587,1879,1972,2151,2222,2250,2448,2300,2324,2487,2201,2179,2073,2345,2202,2019,1961,2017,1967,2019,1867,1797,1827,1556,1523,1457,1781,1579,1157,1142,1123,1245,1215,1230,1060,1225,1120,996,1164,1243,2572,2705,2575,2305,2572,2304,2361,2043,2545,2345,2486,2049,2230,2094,1849,1685,1696,1460,1376,1133,1044,994,918,959,1030,983,1003,889,813,814,879,795,687,594,607,564,552,508,401,383,399,349,257,212,141,39
1,covid-ori-deaths,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23,17,96,317,997,1610,2481,3617,5390,7298,9014,8768,8916,7684,6810,7449,6735,7143,6386,7411,7387,6656,6529,5943,5341,5362,4752,4148,4314,3783,3417,3184,3145,2904,2472,2633,2652,3482,4210,5218,5467,6656,6206,5834,6405,7676,16156,15801,13649,12282,15998,18213,25825,29423,44030,45794,45059,34769,31032,27025,20908,18521,17163,14255,13885,12478,11519,11346,9060,7310,6206,5332,4992,3994,3960,3447,3593,3090,2813,2479,2329,2106,1874,1742,1546,1176,989,829,621,414,185,66
2,covid-ori-recovered,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27,43,153,771,2486,3565,4137,5701,7776,9878,11070,11602,12984,11798,11011,12857,13153,14371,12684,14305,14171,12968,12311,11369,10426,10669,9898,8947,8840,7990,7157,7047,6611,6359,5929,6001,6090,8289,9750,11484,11639,12858,11610,10739,12059,13669,29140,27395,25288,23895,29031,32666,41870,44895,60991,60526,59048,46730,43451,40886,34675,33278,32604,28980,29064,27728,27156,26211,22717,19360,16274,14788,12929,10548,9352,7790,8232,6674,6052,4794,4551,4134,3830,3412,2942,2149,2019,1643,1257,855,357,76
3,covid-pred-recovered,32.0,50.0,53.0,70.0,60.0,44.0,51.0,48,85,115,401,1384,2278,3661,4608,4812,5429,5706,5507,5862,5613,5295,6190,5731,6124,5509,6065,6252,5811,5810,5885,5633,5627,4949,4851,5028,4833,4571,4226,4212,3884,3649,3510,3570,3723,3599,3766,2914,3334,2800,2529,3099,3211,6965,7124,6547,6042,6511,6051,5571,4657,5322,5017,5072,4270,4540,4711,4215,3984,3916,3434,3136,2718,2616,2535,2347,2452,2547,2654,2612,2402,2364,2001,2511,2370,1753,1471,1489,1365,1247,1294,1132,1032,1019,874,663,484,260,43


In [43]:
pd_to_plot.set_index('attr').T.plot(kind='bar', stacked=True,
          colormap=ListedColormap(sns.color_palette()), width=0.5,
          figsize=(200,70))
plt.xticks(fontsize=80, rotation=45)
plt.yticks(fontsize=120)
plt.legend(fontsize=150)
axes = plt.gca()
axes.set_ylim([0,115000])
plt.grid(color='gray', linestyle='-', linewidth=2)
plt.title(label="Evolution of cases - from jan/2020 to dez/2021 - BR", loc='center', fontsize=200)
plt.show()